<a href="https://colab.research.google.com/github/epalosh/SEES-2022/blob/main/Cropland_In_Texas-California-CentralCornBeltPlains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome! This notebook, we will be exporing cropland plots in Texas. Let's go!

In [1]:
import ee

In [2]:
#Authenticate and Initialize Earth Engine!
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=SN7cm3Ue2XJChsTPkosY58SRmkFB0AejWYYfPMZ-6pE&tc=b8LL2H-7c1MJZf5l2fI2krghIS7hbNojDAANxMzvQ5U&cc=k6It88C0zr6BE3R2p2bOw-vACufN6AQXG_TQ6S9fX08

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qjg04mMc8tE2kEeKqWjVhfFBfQjZSRx31btefgL4TQbEnQQJRzp25Y

Successfully saved authorization token.


In [3]:
# Installs geemap package
import subprocess

try:
    import geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap
    print("geemap is now installed, imported and ready to use in Colab") 

geemap package not installed. Installing ...
geemap is now installed, imported and ready to use in Colab


To begin this plot, I create a variable to store the location. In this example, I will be clipping the plot to Texas only by utilizing the TIGER dataset.

Define the feature collection and filter for the desired parameter. In this case, I filter for the state "Texas."

In [4]:
Texas = ee.FeatureCollection('TIGER/2018/States') \
    .filter(ee.Filter.eq("NAME", "Texas"))

Add some more!

In [5]:
California = ee.FeatureCollection('TIGER/2018/States') \
    .filter(ee.Filter.eq("NAME", "California"))

CCornBeltPlains = ee.FeatureCollection('EPA/Ecoregions/2013/L3') \
    .filter(ee.Filter.eq("na_l3code", "8.2.3"))

TravisCounty = ee.FeatureCollection("TIGER/2018/Counties") \
    .filter(ee.Filter.eq("COUNTYFP", "453"))

CountiesInTexas = ee.FeatureCollection('TIGER/2018/Counties') \
    .filter(ee.Filter.eq("STATEFP", "48"))

Next, I will import the data we are looking to map and name it "image." I am utilizing the USDA NASS Cropland Data Layers dataset.

In [6]:
image = ee.ImageCollection("USDA/NASS/CDL")

Next, I'm taking the sum of two date ranges. The ranges span the year of 2021. Then, I filter the image by this sum.

In [7]:
sum1 = ee.Filter.date('2021-01-01','2021-06-30')
sum2 = ee.Filter.date('2021-06-30','2021-12-31')

SumFilter = ee.Filter.Or(sum1, sum2)

allsum = image.filter(SumFilter)

Create a variable called composite that reduces the collection of images.

first() selects the first image from the collection (reduces it to a single image)

clip() clips the image to a defined area.

paint() paints something on the map. In this case, it is the county lines.

In [8]:
composite = allsum \
            .first()

TX = composite.clip(Texas)
CA = composite.clip(California)
CCBP = composite.clip(CCornBeltPlains)
TC = composite.clip(TravisCounty)

CIT = ee.Image().paint(CountiesInTexas, 0, 1)

Finally, map it!!

In [9]:
Map = geemap.Map()   
cropLandcover = image.select('cropland') #Select the band 'cropland'
Map.setCenter(-97.55, 31.71, 4) #set the center point and zoom values

Map.addLayer(TX.select("cropland"), {}, 'Texas') #add the layer of Texas
Map.addLayer(CA.select("cropland"), {}, 'California') #add the layer of California
Map.addLayer(CCBP.select("cropland"), {}, 'Central Corn Belt Plains') #add the layer of the Central Corn Belt Plains Region
Map.addLayer(TC.select("cropland"), {}, 'Travis County') #add the layer of Travis County, TX
Map.addLayer(CIT, {}, 'Counties In Texas') #Add county boundries in Texas

Map.add_legend(builtin_legend='USDA/NASS/CDL') #Add a legend to identify crop types

Map

Map(center=[31.71, -97.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…